# DQN Algorithm test

In [10]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import os
import sys
import warnings

import numpy as np
import torch

ROOT_FOLDER = os.path.join(".", "..")
if ROOT_FOLDER not in sys.path:
    sys.path.insert(0, ROOT_FOLDER)


from dataset import RegexDataset
from environment import Environment

warnings.filterwarnings("ignore")
logging.disable(logging.WARNING)
torch.backends.cudnn.deterministic = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

## Dataset

In [16]:
dataset = RegexDataset(["a2d", "2bb"], r"\d+")
data_iter = dataset.create_iterator()

for i in range(10):
    print(next(data_iter))

('a2d', [0, 1, 0], 1)
('2bb', [1, 0, 0], 1)
('a2d', [0, 1, 0], 1)
('2bb', [1, 0, 0], 1)
('2bb', [1, 0, 0], 1)
('a2d', [0, 1, 0], 1)
('2bb', [1, 0, 0], 1)
('a2d', [0, 1, 0], 1)
('a2d', [0, 1, 0], 1)
('2bb', [1, 0, 0], 1)


## Environment

In [20]:
env = Environment(dataset)

env.action_space

101

In [23]:
state = env.reset()
for _ in range(10):
    action = np.random.randint(env.action_space)
    print(env.step(action))

(array([ 75, 101, 101, 101, 101]), np.int64(-10302), False)
(array([ 75,  64, 101, 101, 101]), np.int64(-10103), False)
(array([ 75,  64,   5, 101, 101]), np.int64(-10104), False)
(array([ 75,  64,   5,  98, 101]), np.int64(-10105), False)


error: unknown extension ?\d at position 1